# Ray RLlib Multi-Armed Bandits - Market Bandit Example

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

Now that we've learned about multi-armed bandits and methods for optimizing rewards, let's look at real-world applications, starting with a stock market example. We'll also learn a little more about configuring RLlib trainers.

We'll load a dataset derived from this [NYU Stern table](http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html) that shows returns for nearly a century of market data, including dividends and adjustments for inflation. The `market.tsv` file in this folder contains the data.

In [1]:
import pandas as pd
import numpy as np
import os, sys

In [2]:
# Some properties we'll need:
DEFAULT_MAX_INFLATION = 100.0
DEFAULT_TICKERS = ["sp500", "t.bill", "t.bond", "corp"]
DEFAULT_DATA_FILE = os.path.abspath(os.path.curdir) + '/market.tsv'  # full path

In [3]:
def load_market_data (file_name):
    with open(file_name, "r") as f:
        return pd.read_table(f)

In [4]:
df = load_market_data(DEFAULT_DATA_FILE)
df

,year,inflation,sp500,t.bill,t.bond,corp
0,1928,-1.15,45.49,4.28,2.01,4.42
1,1929,0.00,-8.30,3.16,4.20,3.02
2,1930,-2.67,-23.07,7.42,7.41,3.30
3,1931,-8.93,-38.33,12.34,7.00,-7.41
4,1932,-10.30,1.85,12.68,21.28,37.78
...,...,...,...,...,...,...
87,2015,0.12,1.26,-0.07,1.16,-0.82
88,2016,1.26,10.38,-0.93,-0.56,8.99
89,2017,2.13,19.07,-1.17,0.66,7.44
90,2018,2.44,-6.51,-0.49,-2.40,-5.08


As you can see the data spans 92 years, from 1928 to 2019. The columns represent:
  * the year
  * inflation rate at the time
  * [S&P500](https://en.wikipedia.org/wiki/S%26P_500_Index) (composite stock index)
  * [Treasury Bills](https://www.investopedia.com/terms/t/treasurybill.asp) (short-term gov bonds)
  * [Treasury Bonds](https://www.investopedia.com/terms/t/treasurybond.asp) (long-term gov bonds)
  * [Moody's Baa Corporate Bonds](https://en.wikipedia.org/wiki/Moody%27s_Investors_Service#Moody's_credit_ratings) (moderate risk)

In [5]:
df.describe()

,year,inflation,sp500,t.bill,t.bond,corp
count,92.00000,92.000000,92.000000,92.000000,92.000000,92.000000
mean,1973.50000,3.041957,8.413261,0.434239,2.166413,4.216630
std,26.70206,3.803579,19.619605,3.573035,8.126432,8.625809
min,1928.00000,-10.300000,-38.900000,-12.050000,-14.570000,-14.850000
25%,1950.75000,1.415000,-2.740000,-1.185000,-2.620000,-1.322500
50%,1973.50000,2.750000,10.515000,0.590000,1.070000,3.910000
75%,1996.25000,4.275000,20.622500,2.117500,7.037500,9.287500
max,2019.00000,14.390000,58.200000,12.680000,25.140000,37.780000


"Corp" refers to corporate bonds.

## Analysis of the Data

What are the worst case and best case scenarios? In other words, if one could predict the future market performance, what are the possible ranges of total failure vs. total success over the past century? By "total", we mean what if you had all your money in a given year invested in the worst performing _sector_ (S&P500 or T bills or ...) or the best performing sector for that year.

In [6]:
n_years = len(df)
min_list = []
max_list = []

for i in range(n_years):
    row = df.iloc[i, 2:]
    min_list.append(min(row))
    max_list.append(max(row))
    
print("{:5.2f}% worst case annualized".format(sum(min_list) / n_years))
print("{:5.2f}% best case annualized".format(sum(max_list) / n_years))

-5.64% worst case annualized
15.18% best case annualized


In [7]:
sys.path.append('../..')
from util.line_plots import plot_line, plot_line_with_stddev, plot_between_lines
from bokeh_util import plot_cumulative_regret

In [8]:
import bokeh
bokeh.io.reset_output()
bokeh.io.output_notebook()

Loading BokehJS ...

In [9]:
min_max = pd.DataFrame.from_dict({'year': df['year'], 'min':min_list, 'max':max_list})
min_max

,year,min,max
0,1928,2.01,45.49
1,1929,-8.30,4.20
2,1930,-23.07,7.42
3,1931,-38.33,12.34
4,1932,1.85,37.78
...,...,...,...
87,2015,-0.82,1.26
88,2016,-0.93,10.38
89,2017,-1.17,19.07
90,2018,-6.51,-0.49


In [10]:
plot_between_lines(min_max, x_col='year', lower_col='min', upper_col='max', 
                   title='Best to Worst', x_axis_label='year', y_axis_label='%')

## Defining an Environment

Now let's define a Gym environment so that we can train a contextual bandit to optimize annual investments over that period.

In [11]:
import gym
from gym.spaces import Discrete, Box
from gym.utils import seeding
import numpy as np
import random

This is the bandit we'll use to represent the market "environment".

In [12]:
class MarketBandit (gym.Env):
    
    def __init__ (self, config={}):
        self.max_inflation = config.get('max-inflation', DEFAULT_MAX_INFLATION)
        self.tickers = config.get('tickers', DEFAULT_TICKERS)
        self.data_file = config.get('data-file', DEFAULT_DATA_FILE)
        print(f"MarketBandit: max_inflation: {self.max_inflation}, tickers: {self.tickers}, data file: {self.data_file} (config: {config})")

        self.action_space = Discrete(4)
        self.observation_space = Box(
            low  = -self.max_inflation,
            high =  self.max_inflation,
            shape=(1, )
        )
        self.df = load_market_data(self.data_file)
        self.cur_context = None


    def reset (self):
        self.year = self.df["year"].min()
        self.cur_context = self.df.loc[self.df["year"] == self.year]["inflation"][0]
        self.done = False
        self.info = {}

        return [self.cur_context]


    def step (self, action):
        if self.done:
            reward = 0.
            regret = 0.
        else:
            row = self.df.loc[self.df["year"] == self.year]

            # calculate reward
            ticker = self.tickers[action]
            reward = float(row[ticker])

            # calculate regret
            max_reward = max(map(lambda t: float(row[t]), self.tickers))
            regret = round(max_reward - reward)

            # update the context
            self.cur_context = float(row["inflation"])

            # increment the year
            self.year += 1

            if self.year >= self.df["year"].max():
                self.done = True

        context = [self.cur_context]
        #context = self.observation_space.sample()

        self.info = {
            "regret": regret,
            "year": self.year
        }
         
        return [context, reward, self.done, self.info]


    def seed (self, seed=None):
        """Sets the seed for this env's random number generator(s).
        Note:
            Some environments use multiple pseudorandom number generators.
            We want to capture all such seeds used in order to ensure that
            there aren't accidental correlations between multiple generators.
        Returns:
            list<bigint>: Returns the list of seeds used in this env's random
              number generators. The first value in the list should be the
              "main" seed, or the value which a reproducer should pass to
              'seed'. Often, the main seed equals the provided 'seed', but
              this won't be true if seed=None, for example.
        """
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

Let's see it in action:

In [13]:
bandit = MarketBandit()
bandit.reset()

for i in range(10):
    action = bandit.action_space.sample()
    obs = bandit.step(action)
    print(action, obs)

MarketBandit: max_inflation: 100.0, tickers: ['sp500', 't.bill', 't.bond', 'corp'], data file: /Users/deanwampler/projects/anyscale/academy/academy-git/ray-rllib/multi-armed-bandits/market.tsv (config: {})
1 [[-1.15], 4.28, False, {'regret': 41, 'year': 1929}]
3 [[0.0], 3.02, False, {'regret': 1, 'year': 1930}]
0 [[-2.67], -23.07, False, {'regret': 30, 'year': 1931}]
0 [[-8.93], -38.33, False, {'regret': 51, 'year': 1932}]
1 [[-10.3], 12.68, False, {'regret': 25, 'year': 1933}]
1 [[-5.19], 6.49, False, {'regret': 52, 'year': 1934}]
2 [[3.48], 4.33, False, {'regret': 10, 'year': 1935}]
1 [[2.55], -2.33, False, {'regret': 45, 'year': 1936}]
1 [[1.03], -0.85, False, {'regret': 31, 'year': 1937}]
1 [[3.73], -3.33, False, {'regret': 1, 'year': 1938}]


/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


We can use this environment in a kind of *monte carlo simulation* to measure a baseline for what the rewards would be over a long period if you always used a random action.

In [14]:
done = 1
reward_list = []
iterations = 10000 #50000

for i in range(iterations):
    if done == 1:
        bandit.reset()

    action = bandit.action_space.sample()
    obs = bandit.step(action)
    context, reward, done, info = obs
    reward_list.append(reward)
    #print(action, context, reward, done, info)

In [15]:
df_mc = pd.DataFrame(reward_list, columns=["reward"])
df_mc.mean()

reward    3.838733
dtype: float64

Depending on the number of iterations, you'll probably get a value approaching 3.75% as a baseline for random actions. That's more than the -5.64% worst case and must less than 15.18% best case for the reward!

In [16]:
plot_line(df_mc, x_col='index', y_col='reward', title='Reward Over Time')

([image](../../images/rllib/MarketReward-Random.png))

Yes, it looks quite random...

## Training a policy in RLlib

Now let's train a policy using our contextual bandit, specifically using _Linear Thompson Sampling_ in RLlib. Hopefully it will do better than the random results we just computed!

Recall in the `__init__()` method for `MarketBandit` that we set some parameters from the passed in `config` object (with defaults). We don't construct this explicitly ourselves. Rather, RLlib will do this. So, we need to construct the canonical `config` object we want to use. To do this, we use the idioms shown in the next several cells:

In [17]:
from ray.rllib.agents.trainer import with_base_config, with_common_config
from ray.rllib.contrib.bandits.agents.lin_ts import TS_CONFIG
from ray.rllib.contrib.bandits.agents.lin_ts import LinTSTrainer
import ray

Initialize Ray as required:

In [20]:
!../../tools/start-ray.sh --check --verbose

INFO: Ray is already running.


In [21]:
ray.init(address='auto', ignore_reinit_error=True)

2020-06-13 08:31:05,901	WARNING worker.py:809 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.


{'node_ip_address': '192.168.1.149',
 'raylet_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:15832',
 'object_store_address': '/tmp/ray/session_2020-06-12_08-58-38_626987_40764/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-12_08-58-38_626987_40764/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-06-12_08-58-38_626987_40764'}

We need a custom config object with our parameters for `MarketBandit`. We do this building on the default `TS_CONFIG` object for _LinTS_:

In [22]:
market_config = with_base_config(TS_CONFIG, {
    "env":           MarketBandit,
    'max-inflation': DEFAULT_MAX_INFLATION,
    'tickers':       DEFAULT_TICKERS,
    'data-file':     DEFAULT_DATA_FILE
})

stop = {
    "training_iteration": 100
}

Also, we'll define a custom trainer, which builds on the `LinTSTrainer`, with "updates". Note that it's the first argument we'll pass to `tune.run()` in the following cell. When all we need is `LinTSTrainer`, as is, and no extra custom config settings, we can just pass the string `contrib/LinTS` to `tune.run()`.  

In [23]:
MarketLinTSTrainer = LinTSTrainer.with_updates(
    name="MarketLinTSTrainer",
    default_config=market_config,      # Will be merged with Trainer.COMMON_CONFIG (rllib/agent/trainer.py)
    #default_policy=[somePolicyClass]  # If we had a policy...
)

In [24]:
analysis = ray.tune.run(
    MarketLinTSTrainer,
    config=market_config,
    stop=stop,
    num_samples=3,    
    checkpoint_at_end=True,
    verbose=2,              # Change to 0 or 1 to reduce the output.
    ray_auto_init=False,    # Don't allow Tune to initialize Ray.
)

Trial name,status,loc
MarketLinTSTrainer_MarketBandit_00000,RUNNING,
MarketLinTSTrainer_MarketBandit_00001,PENDING,
MarketLinTSTrainer_MarketBandit_00002,PENDING,


(pid=54752) 2020-06-13 08:31:24,230	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=54752) 2020-06-13 08:31:24,235	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=54752) /Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=54752)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=54752) 2020-06-13 08:31:24,249	INFO trainable.py:217 -- Getting current IP.
(pid=54752) 2020-06-13 08:31:24,250	WARNING util.py:37 -- Install gputil for GPU system monitoring.
(pid=54752) MarketBandit: max_inflation: 100.0, tickers: ['sp500', 't.bill', 't.bond', 'corp'], data file: /Users/deanwampler/projects/anyscale/academy/academy-git/ray-rllib/multi-armed-bandits/market.tsv (config: {})
(pid=72865) MarketBandit: max_infl

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,RUNNING,,,,,
MarketLinTSTrainer_MarketBandit_00001,RUNNING,,,,,
MarketLinTSTrainer_MarketBandit_00002,RUNNING,192.168.1.149:72866,1,0.211371,100,554.32


Result for MarketLinTSTrainer_MarketBandit_00000:
  custom_metrics: {}
  date: 2020-06-13_08-31-24
  done: false
  episode_len_mean: 91.0
  episode_reward_max: 238.16000000000003
  episode_reward_mean: 238.16000000000003
  episode_reward_min: 238.16000000000003
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: e231ea90d740427cab1a54b97eecc390
  experiment_tag: '0'
  grad_time_ms: 0.304
  hostname: DWAnyscaleMBP.local
  info:
    grad_time_ms: 0.304
    learner:
      cumulative_regret: 1322.0
      update_latency: 0.00019407272338867188
    num_steps_sampled: 100
    num_steps_trained: 100
    opt_peak_throughput: 3294.042
    opt_samples: 1.0
    sample_peak_throughput: 582.526
    sample_time_ms: 1.717
    update_time_ms: 0.001
  iterations_since_restore: 1
  learner:
    cumulative_regret: 1322.0
    update_latency: 0.00019407272338867188
  node_ip: 192.168.1.149
  num_healthy_workers: 0
  num_steps_sampled: 100
  num_steps_trained: 100
  off_policy_estimator: {}
  opt_pe

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,RUNNING,192.168.1.149:54752,18,4.98009,1800,461.842
MarketLinTSTrainer_MarketBandit_00001,RUNNING,192.168.1.149:72865,18,4.96412,1800,511.924
MarketLinTSTrainer_MarketBandit_00002,RUNNING,192.168.1.149:72866,18,5.02767,1800,627.763


...

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,RUNNING,192.168.1.149:54752,81,25.0801,8100,556.257
MarketLinTSTrainer_MarketBandit_00001,RUNNING,192.168.1.149:72865,80,24.8734,8000,569.882
MarketLinTSTrainer_MarketBandit_00002,RUNNING,192.168.1.149:72866,80,25.0045,8000,611.861


Result for MarketLinTSTrainer_MarketBandit_00001:
  custom_metrics: {}
  date: 2020-06-13_08-31-50
  done: false
  episode_len_mean: 91.0
  episode_reward_max: 790.8899999999998
  episode_reward_mean: 570.4105617977527
  episode_reward_min: 268.11000000000007
  episodes_this_iter: 2
  episodes_total: 89
  experiment_id: 80e5532cb0b248d99933b7e4fff9be95
  experiment_tag: '1'
  grad_time_ms: 0.589
  hostname: DWAnyscaleMBP.local
  info:
    grad_time_ms: 0.589
    learner:
      cumulative_regret: 70929.0
      update_latency: 0.0002799034118652344
    num_steps_sampled: 8100
    num_steps_trained: 8100
    opt_peak_throughput: 1697.137
    opt_samples: 1.0
    sample_peak_throughput: 438.827
    sample_time_ms: 2.279
    update_time_ms: 0.002
  iterations_since_restore: 81
  learner:
    cumulative_regret: 70929.0
    update_latency: 0.0002799034118652344
  node_ip: 192.168.1.149
  num_healthy_workers: 0
  num_steps_sampled: 8100
  num_steps_trained: 8100
  off_policy_estimator: {}
  op

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,RUNNING,192.168.1.149:54752,95,29.896,9500,565.182
MarketLinTSTrainer_MarketBandit_00001,RUNNING,192.168.1.149:72865,95,30.0285,9500,578.665
MarketLinTSTrainer_MarketBandit_00002,RUNNING,192.168.1.149:72866,94,29.8141,9400,612.354


Result for MarketLinTSTrainer_MarketBandit_00002:
  custom_metrics: {}
  date: 2020-06-13_08-31-55
  done: false
  episode_len_mean: 91.0
  episode_reward_max: 816.27
  episode_reward_mean: 613.6746
  episode_reward_min: 423.94000000000005
  episodes_this_iter: 1
  episodes_total: 104
  experiment_id: 502a66c6c16c4e759e4fed19ee71f191
  experiment_tag: '2'
  grad_time_ms: 0.618
  hostname: DWAnyscaleMBP.local
  info:
    grad_time_ms: 0.618
    learner:
      cumulative_regret: 79102.0
      update_latency: 0.0004029273986816406
    num_steps_sampled: 9500
    num_steps_trained: 9500
    opt_peak_throughput: 1618.423
    opt_samples: 1.0
    sample_peak_throughput: 371.862
    sample_time_ms: 2.689
    update_time_ms: 0.002
  iterations_since_restore: 95
  learner:
    cumulative_regret: 79102.0
    update_latency: 0.0004029273986816406
  node_ip: 192.168.1.149
  num_healthy_workers: 0
  num_steps_sampled: 9500
  num_steps_trained: 9500
  off_policy_estimator: {}
  opt_peak_throughput: 

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,TERMINATED,,100,31.6796,10000,573.394
MarketLinTSTrainer_MarketBandit_00001,TERMINATED,,100,31.8161,10000,580.311
MarketLinTSTrainer_MarketBandit_00002,TERMINATED,,100,31.9002,10000,609.767


## Analyzing the results

Let's analyze the rewards and cumulative regrets of these trials.

In [25]:
df_ts = pd.DataFrame()

for key, df_trial in analysis.trial_dataframes.items():
    df_ts = df_ts.append(df_trial, ignore_index=True)
    
df_ts.head()

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_steps_trained,num_steps_sampled,sample_time_ms,grad_time_ms,update_time_ms,...,info/update_time_ms,info/opt_peak_throughput,info/sample_peak_throughput,info/opt_samples,learner/cumulative_regret,learner/update_latency,perf/cpu_util_percent,perf/ram_util_percent,info/learner/cumulative_regret,info/learner/update_latency
0,238.16,238.16,238.160000,91.0,1,100,100,1.717,0.304,0.001,...,0.001,3294.042,582.526,1.0,1322.0,0.000194,21.0,67.9,1322.0,0.000194
1,261.35,238.16,249.755000,91.0,1,200,200,1.757,0.342,0.002,...,0.002,2924.083,569.267,1.0,2514.0,0.000171,54.6,67.9,2514.0,0.000171
2,295.45,238.16,264.986667,91.0,1,300,300,1.744,0.328,0.001,...,0.001,3050.181,573.384,1.0,3608.0,0.000181,NaN,NaN,3608.0,0.000181
3,466.72,238.16,315.420000,91.0,1,400,400,13.278,2.189,0.002,...,0.002,456.901,75.313,1.0,4707.0,0.002375,NaN,NaN,4707.0,0.002375
4,466.72,238.16,328.280000,91.0,1,500,500,2.601,0.464,0.002,...,0.002,2154.571,384.425,1.0,5674.0,0.000207,0.0,67.9,5674.0,0.000207


In [26]:
rewards = df_ts \
    .groupby("num_steps_trained")["episode_reward_mean"] \
    .aggregate(["mean", "max", "min", "std"])

rewards

,mean,max,min,std
num_steps_trained,,,,
100,408.310000,554.320000,238.160000,159.456392
200,371.015000,513.010000,249.755000,132.846731
300,394.255556,543.803333,264.986667,140.510172
400,413.629167,557.517500,315.420000,127.348889
500,438.542000,586.884000,328.280000,133.441268
...,...,...,...,...
9600,586.352667,613.202000,566.811200,24.043300
9700,586.445233,611.946300,567.807300,22.855848
9800,586.712133,611.223200,569.502700,21.797601


In [27]:
plot_line_with_stddev(rewards, x_col='num_steps_trained', y_col='mean', stddev_col='std', 
                      title='Rewards vs. Steps', x_axis_label='step', y_axis_label='reward')

([image](../../images/rllib/Market-Bandit-Rewards-vs-Steps.png))

The rewards reach what appears to be nearly optimzal by 3000 steps, then shows some slow improvement above 8000.

In [28]:
regrets = df_ts \
    .groupby("num_steps_trained")["learner/cumulative_regret"] \
    .aggregate(["mean", "max", "min", "std"])

regrets

,mean,max,min,std
num_steps_trained,,,,
100,1138.000000,1322.0,988.0,169.575942
200,2227.000000,2514.0,1886.0,317.463384
300,3242.000000,3608.0,2745.0,446.164768
400,4238.000000,4707.0,3561.0,600.647151
500,5099.666667,5674.0,4319.0,700.668490
...,...,...,...,...
9600,83217.333333,85721.0,79869.0,3016.052442
9700,84084.333333,86464.0,80777.0,2954.812402
9800,84998.666667,87303.0,81645.0,2971.374486


In [29]:
plot_cumulative_regret(regrets)

([image](../../images/rllib/Market-Bandit-Cumulative-Regret.png))

## Evaluating the Trained Policy

Overall, how well did the trained policy perform? The results should be better than random, but less than the best case.

In [30]:
print("{:5.2f}% optimized return annualized".format(max(rewards["mean"]) / n_years))

 6.39% optimized return annualized


That's better than the random action baseline of 3.75%, but no where near the best case scenario of 15.18% return. Hence, our regrets grow...

Note that investing solely in the S&P stock index which would have produced better than 8% return over that period -- that is, if one could wait 92 years. However, investing one's entire portfolio into stocks can become quite a risky policy in the short-term, so we were exploring how to balance a portfolio given only limited information.

In any case, the contextual bandit performed well considering that it could only use *inflation* for the context of its decisions, and could only take actions once each year.

## Exercise 1

Try using a `LinUCBTrainer`-based trainer. How does the annualized return compare?

---

## Extra - Restoring from a Checkpoint

In the previous lesson, [05 Thompson Sampling](05-Thompson-Sampling.ipynb), we showed how to restore a trainer from a checkpoint, but almost "in passing". Let's use this feature again, this time with our custom trainer class `MarketLinTSTrainer`.

In [31]:
trial = analysis.trials[0]
path = trial.checkpoint.value
print(f'checkpoint_path: {path}')

checkpoint_path: /Users/deanwampler/ray_results/MarketLinTSTrainer/MarketLinTSTrainer_MarketBandit_0_2020-06-13_08-31-18zdww1blg/checkpoint_100/checkpoint-100


In [32]:
trainer = MarketLinTSTrainer(market_config)  # create instance and then restore from checkpoint
trainer.restore(path)

2020-06-13 08:35:29,486	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-06-13 08:35:29,500	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2020-06-13 08:35:29,566	INFO trainable.py:217 -- Getting current IP.
2020-06-13 08:35:29,567	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020-06-13 08:35:29,572	INFO trainable.py:217 -- Getting current IP.
2020-06-13 08:35:29,572	INFO trainable.py:423 -- Restored on 192.168.1.149 from checkpoint: /Users/deanwampler/ray_results/MarketLinTSTrainer/MarketLinTSTrainer_MarketBandit_0_2020-06-13_08-31-18zdww1blg/checkpoint_100/checkpoint-100
2020-06-13 08:35:29,5

MarketBandit: max_inflation: 100.0, tickers: ['sp500', 't.bill', 't.bond', 'corp'], data file: /Users/deanwampler/projects/anyscale/academy/academy-git/ray-rllib/multi-armed-bandits/market.tsv (config: {})


Get model to plot arm weights distribution

In [33]:
model = trainer.get_policy().model
means = [model.arms[i].theta.numpy() for i in range(3)]
covs = [model.arms[i].covariance.numpy() for i in range(3)]
means, covs, model.arms[0].theta.numpy()

([array([1.3072101], dtype=float32),
  array([-0.3498547], dtype=float32),
  array([-0.5549358], dtype=float32)],
 [array([[7.4439145e-06]], dtype=float32),
  array([[3.406569e-05]], dtype=float32),
  array([[4.1243744e-05]], dtype=float32)],
 array([1.3072101], dtype=float32))

(pid=74572) 2020-06-13 09:26:26,394	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=74572) 2020-06-13 09:26:26,398	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=74572) 2020-06-13 09:26:26,409	INFO trainable.py:217 -- Getting current IP.
(pid=74572) 2020-06-13 09:26:26,409	WARNING util.py:37 -- Install gputil for GPU system monitoring.
(pid=74570) 2020-06-13 09:29:54,612	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=74570) 2020-06-13 09:29:54,616	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=74570) 2020-06-13 09:29:54,625	INFO trainable.py:217 -- Getting current IP.
(pid=74570) 2020-06-13 09:29:54,626	WARNING util.py:37 -- Install gputil for GPU system monitoring.
(pid=74571) 2020-0

A final note; when you checkpoint the model, it will change how the training performs in this notebook, if you rerun the training! So, when doing experiments, be sure you are starting from scratch when that is desired!